In [1]:
import pandas as pd

In [2]:
train=pd.read_csv('lightGBM_out_of_fold_train_predictions.csv')

In [3]:
train.shape

(307511, 3)

In [4]:
train.head()

,SK_ID_CURR,fold_id,lightGBM_prediction
0,100002,0,0.304156
1,100009,0,0.005594
2,100011,0,0.049608
3,100014,0,0.045775
4,100017,0,0.018538


In [5]:
test=pd.read_csv('lightGBM_out_of_fold_test_predictions.csv')

In [6]:
test.shape

(243720, 3)

In [7]:
test.head()

,SK_ID_CURR,fold_id,lightGBM_prediction
0,100001,0,0.046421
1,100005,0,0.113959
2,100013,0,0.036334
3,100028,0,0.046933
4,100038,0,0.159083


In [8]:
set(train['fold_id'])

{0, 1, 2, 3, 4}

In [9]:
prediction=pd.read_csv('lightGBM_test_predictions_rank_mean.csv')

In [10]:
prediction.shape

(48744, 2)

In [11]:
from sklearn.linear_model import LogisticRegression
from scipy.special import expit, logit
from sklearn.metrics import roc_auc_score

In [13]:
train_org=pd.read_csv('/DATACENTER2/scy/Home_Credit/raw_data/application_train.csv')

In [14]:
train_org=train_org[['SK_ID_CURR', 'TARGET']]

In [15]:
train_org.head()

,SK_ID_CURR,TARGET
0,100002,1
1,100003,0
2,100004,0
3,100006,0
4,100007,0


In [16]:
train=train.merge(train_org, on='SK_ID_CURR', how='left')

In [17]:
train.head()

,SK_ID_CURR,fold_id,lightGBM_prediction,TARGET
0,100002,0,0.304156,1
1,100009,0,0.005594,0
2,100011,0,0.049608,0
3,100014,0,0.045775,0
4,100017,0,0.018538,0


In [18]:
train.isnull().sum()

SK_ID_CURR             0
fold_id                0
lightGBM_prediction    0
TARGET                 0
dtype: int64

In [19]:
X_train=train[['lightGBM_prediction']]
y_train=train['TARGET']

In [20]:
X_train.shape

(307511, 1)

In [21]:
y_train.shape

(307511,)

In [22]:
stack_model = LogisticRegression()
stack_model.fit(X_train, y_train)
stack_model.coef_

array([[7.93783291]])

In [23]:
test0=test[test['fold_id']==0][['lightGBM_prediction']]
test1=test[test['fold_id']==1][['lightGBM_prediction']]
test2=test[test['fold_id']==2][['lightGBM_prediction']]
test3=test[test['fold_id']==3][['lightGBM_prediction']]
test4=test[test['fold_id']==4][['lightGBM_prediction']]

In [24]:
pred0=stack_model.predict_proba(test0)[:,1]
pred1=stack_model.predict_proba(test1)[:,1]
pred2=stack_model.predict_proba(test2)[:,1]
pred3=stack_model.predict_proba(test3)[:,1]
pred4=stack_model.predict_proba(test4)[:,1]

In [25]:
sub = pd.DataFrame()
sub['SK_ID_CURR'] = test[test['fold_id']==0]['SK_ID_CURR']

In [26]:
sub['pred0']=pred0
sub['pred1']=pred1
sub['pred2']=pred2
sub['pred3']=pred3
sub['pred4']=pred4

In [34]:
sub['TARGET']=(sub['pred0']+sub['pred1']+sub['pred2']+sub['pred3']+sub['pred4'])/5

In [36]:
import numpy as np
np.min(sub['TARGET'])

0.034995608391432995

In [37]:
np.max(sub['TARGET'])

0.94301124000203251

In [38]:
sub[['SK_ID_CURR', 'TARGET']].to_csv('sub_stack.csv', index= False)

In [ ]:
################################################################
############################   XGBOOST  ####################################
################################################################

In [3]:
train=pd.read_csv('out_folder_train_prediction_xgb.csv')

In [4]:
train

,train_pred,SK_ID_CURR,TARGET,folder
0,0.035678,100004,0.0,1
1,0.042576,100006,0.0,1
2,0.057968,100008,0.0,1
3,0.055486,100011,0.0,1
4,0.006375,100018,0.0,1
5,0.027167,100023,0.0,1
6,0.082319,100053,0.0,1
7,0.028539,100054,0.0,1
8,0.016324,100060,0.0,1
9,0.019718,100062,0.0,1


In [5]:
test=pd.read_csv('out_folder_test_prediction_xgb.csv')

In [6]:
test.shape

(243720, 3)

In [7]:
test.head()

,SK_ID_CURR,test_pred,folder
0,100001,0.029790,1
1,100005,0.237822,1
2,100013,0.027966,1
3,100028,0.035709,1
4,100038,0.181272,1


In [9]:
set(train['folder'])

{1, 2, 3, 4, 5}

In [10]:
from sklearn.linear_model import LogisticRegression
from scipy.special import expit, logit
from sklearn.metrics import roc_auc_score

In [12]:
train_org=pd.read_csv('/DATACENTER2/scy/Home_Credit/raw_data/application_train.csv')

In [13]:
train_org=train_org[['SK_ID_CURR', 'TARGET']]

In [14]:
train_org.head()

,SK_ID_CURR,TARGET
0,100002,1
1,100003,0
2,100004,0
3,100006,0
4,100007,0


In [15]:
train=train.merge(train_org, on='SK_ID_CURR', how='left')

In [18]:
train

,train_pred,SK_ID_CURR,TARGET_x,folder,TARGET_y
0,0.035678,100004,0.0,1,0
1,0.042576,100006,0.0,1,0
2,0.057968,100008,0.0,1,0
3,0.055486,100011,0.0,1,0
4,0.006375,100018,0.0,1,0
5,0.027167,100023,0.0,1,0
6,0.082319,100053,0.0,1,0
7,0.028539,100054,0.0,1,0
8,0.016324,100060,0.0,1,0
9,0.019718,100062,0.0,1,0


In [17]:
train.isnull().sum()

train_pred    0
SK_ID_CURR    0
TARGET_x      0
folder        0
TARGET_y      0
dtype: int64

In [19]:
X_train=train[['train_pred']]
y_train=train['TARGET_x']

In [20]:
X_train.shape

(307511, 1)

In [21]:
y_train.shape

(307511,)

In [22]:
stack_model = LogisticRegression()
stack_model.fit(X_train, y_train)
stack_model.coef_

array([[ 8.05201165]])

In [26]:
test1=test[test['folder']==1][['test_pred']]
test2=test[test['folder']==2][['test_pred']]
test3=test[test['folder']==3][['test_pred']]
test4=test[test['folder']==4][['test_pred']]
test5=test[test['folder']==5][['test_pred']]

In [27]:
pred1=stack_model.predict_proba(test1)[:,1]
pred2=stack_model.predict_proba(test2)[:,1]
pred3=stack_model.predict_proba(test3)[:,1]
pred4=stack_model.predict_proba(test4)[:,1]
pred5=stack_model.predict_proba(test5)[:,1]

In [28]:
sub = pd.DataFrame()
sub['SK_ID_CURR'] = test[test['folder']==1]['SK_ID_CURR']

In [29]:
sub['pred1']=pred1
sub['pred2']=pred2
sub['pred3']=pred3
sub['pred4']=pred4
sub['pred5']=pred5

In [30]:
sub['TARGET']=(sub['pred1']+sub['pred2']+sub['pred3']+sub['pred4']+sub['pred5'])/5

In [31]:
import numpy as np
np.min(sub['TARGET'])

0.035115606352529728

In [32]:
np.max(sub['TARGET'])

0.95344132513688362

In [33]:
sub[['SK_ID_CURR', 'TARGET']].to_csv('sub_stack_xgb.csv', index= False)